In [2]:
import numpy as np
import pandas as pd

In [3]:
train_data_eq_prior = []
test_data_eq_prior = []
train_data_noneq_prior = []
test_data_noneq_prior = []

est_mean_eq_prior = [0,0,0,0,0]
est_mean_noneq_prior = [0,0,0,0,0]
est_var_eq_prior = [0,0,0,0,0]
est_var_noneq_prior = [0,0,0,0,0]
est_prior_eq_prior = [0,0,0,0,0]
est_prior_noneq_prior = [0,0,0,0,0]
train_count_eq_prior = [0,0,0,0,0]
test_count_eq_prior = [0,0,0,0,0]
train_count_noneq_prior = [0,0,0,0,0]
test_count_noneq_prior = [0,0,0,0,0]

mean = [0,10,5,20,15]
var = [1,2,4,1.2,7]

data1 = np.random.choice(5, 50000, p=[0.2,0.2,0.2,0.2,0.2])
data2 = np.random.choice(5, 10000, p=[0.2,0.2,0.2,0.2,0.2])
data3 = np.random.choice(5, 50000, p=[0.1,0.25,0.3,0.25,0.1])
data4 = np.random.choice(5, 10000, p=[0.1,0.25,0.3,0.25,0.1])

for i in range(50000):
    train_data_eq_prior.append([np.random.normal(mean[data1[i]],var[data1[i]]), data1[i]])
    train_data_noneq_prior.append([np.random.normal(mean[data3[i]],var[data3[i]]), data3[i]])
    train_count_eq_prior[data1[i]] += 1
    train_count_noneq_prior[data3[i]] += 1
    

for i in range(10000):
    test_data_eq_prior.append([np.random.normal(mean[data2[i]],var[data2[i]]), data2[i]])
    test_data_noneq_prior.append([np.random.normal(mean[data4[i]],var[data4[i]]), data4[i]])
    test_count_eq_prior[data2[i]] += 1
    test_count_noneq_prior[data4[i]] += 1


In [6]:
for i in range(50000):
    est_mean_eq_prior[train_data_eq_prior[i][1]] += train_data_eq_prior[i][0]
    est_mean_noneq_prior[train_data_noneq_prior[i][1]] += train_data_noneq_prior[i][0]
    est_prior_eq_prior[train_data_eq_prior[i][1]] += 1
    est_prior_noneq_prior[train_data_noneq_prior[i][1]] += 1

for i in range(5):
    est_mean_eq_prior[i] /= est_prior_eq_prior[i]
    est_mean_noneq_prior[i] /= est_prior_noneq_prior[i]

for i in range(50000):
    est_var_eq_prior[train_data_eq_prior[i][1]] += (train_data_eq_prior[i][0]-est_mean_eq_prior[train_data_eq_prior[i][1]])*(train_data_eq_prior[i][0]-est_mean_eq_prior[train_data_eq_prior[i][1]])
    est_var_noneq_prior[train_data_noneq_prior[i][1]] += (train_data_noneq_prior[i][0]-est_mean_noneq_prior[train_data_noneq_prior[i][1]])*(train_data_noneq_prior[i][0]-est_mean_noneq_prior[train_data_noneq_prior[i][1]])

for i in range(5):
    est_var_eq_prior[i]/est_prior_eq_prior[i]
    est_var_noneq_prior[i]/est_prior_noneq_prior[i]
    est_prior_eq_prior[i] /= 50000
    est_prior_noneq_prior[i] /= 50000

est_var_eq_prior


[30535.36069120531,
 120527.76602786433,
 489013.2999957265,
 43300.384368430845,
 1449426.9966812413]

In [7]:
# equal priors and variances
def makePred_eq_priors_eq_var(trainData,testData,est_mean):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        if i < 10000:
            res = [abs(testData[i][0]-est_mean[0]),abs(testData[i][0]-est_mean[1]),abs(testData[i][0]-est_mean[2]),abs(testData[i][0]-est_mean[3]),abs(testData[i][0]-est_mean[4])]
            res = res.index(min(res))
            pred_test[i] = res
        res = [abs(trainData[i][0]-est_mean[0]),abs(trainData[i][0]-est_mean[1]),abs(trainData[i][0]-est_mean[2]),abs(trainData[i][0]-est_mean[3]),abs(trainData[i][0]-est_mean[4])]
        res = res.index(min(res))
        pred_train[i] = res
    return pred_train, pred_test

In [8]:
def calcAccuracy(trainLabel_DIST1,count_train,pred_train,testLabel_DIST1,count_test,pred_test):
    train_accuracy = [0,0,0,0,0]
    test_accuracy = [0,0,0,0,0]
    for i in range(50000):
        if i < 10000:
            if testLabel_DIST1[i][1] == pred_test[i]:
                test_accuracy[testLabel_DIST1[i][1]] += 1
        if trainLabel_DIST1[i][1] == pred_train[i]:
            train_accuracy[trainLabel_DIST1[i][1]] += 1

    for i in range(5):
        train_accuracy[i] /= count_train[i]
        test_accuracy[i] /= count_test[i]
    return train_accuracy, test_accuracy

In [9]:
# equal priors and estimated variances
def makePred_eq_priors_est_var(trainData,testData,est_mean,est_var):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        if i < 10000:
            res = [-np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[0]),2)/(2 * est_var[0])),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[1]),2)/(2 * est_var[1])),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[2]),2)/(2 * est_var[2])),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[3]),2)/(2 * est_var[3])),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[4]),2)/(2 * est_var[4]))]
            res = res.index(max(res))
            pred_test[i] = res
        res = [-np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[0]),2)/(2 * est_var[0])),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[1]),2)/(2 * est_var[1])),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[2]),2)/(2 * est_var[2])),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[3]),2)/(2 * est_var[3])),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[4]),2)/(2 * est_var[4]))]
        res = res.index(max(res))
        pred_train[i] = res
    return pred_train, pred_test

In [10]:
# estimated priors and variances
def makePred_est_priors_est_var(trainData,testData,est_mean,est_var,est_prior):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        if i < 10000:
            res = [-np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[0]),2)/(2 * est_var[0])) + np.log(est_prior[0]),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[1]),2)/(2 * est_var[1]))+ np.log(est_prior[1]),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[2]),2)/(2 * est_var[2]))+ np.log(est_prior[2]),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[3]),2)/(2 * est_var[3]))+ np.log(est_prior[3]),
                   -np.log(np.sqrt(est_var[0])) - (np.power((testData[i][0] - est_mean[4]),2)/(2 * est_var[4]))+ np.log(est_prior[4])]
            res = res.index(max(res))
            pred_test[i] = res
        res = [-np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[0]),2)/(2 * est_var[0]))+ np.log(est_prior[0]),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[1]),2)/(2 * est_var[1]))+ np.log(est_prior[1]),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[2]),2)/(2 * est_var[2]))+ np.log(est_prior[2]),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[3]),2)/(2 * est_var[3]))+ np.log(est_prior[3]),
               -np.log(np.sqrt(est_var[0])) - (np.power((trainData[i][0] - est_mean[4]),2)/(2 * est_var[4]))+ np.log(est_prior[4])]
        res = res.index(max(res))
        pred_train[i] = res
    return pred_train, pred_test

In [11]:
# estimated priors and equal variances
def makePred_est_priors_eq_var(trainData,testData,est_mean,est_prior):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        if i < 10000:
            res = [ - np.power((testData[i][0] - est_mean[0]),2) + np.log(est_prior[0]),
                    - np.power((testData[i][0] - est_mean[1]),2)+ np.log(est_prior[1]),
                    - np.power((testData[i][0] - est_mean[2]),2)+ np.log(est_prior[2]),
                    - np.power((testData[i][0] - est_mean[3]),2)+ np.log(est_prior[3]),
                    - np.power((testData[i][0] - est_mean[4]),2)+ np.log(est_prior[4])]
            res = res.index(max(res))
            pred_test[i] = res
        res = [ - np.power((trainData[i][0] - est_mean[0]),2)+ np.log(est_prior[0]),
                - np.power((trainData[i][0] - est_mean[1]),2)+ np.log(est_prior[1]),
                - np.power((trainData[i][0] - est_mean[2]),2)+ np.log(est_prior[2]),
                - np.power((trainData[i][0] - est_mean[3]),2)+ np.log(est_prior[3]),
                - np.power((trainData[i][0] - est_mean[4]),2)+ np.log(est_prior[4])]
        res = res.index(max(res))
        pred_train[i] = res
    return pred_train, pred_test

In [12]:
pred_train, pred_test = makePred_eq_priors_eq_var(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior)
result1 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train,test_data_eq_prior,test_count_eq_prior,pred_test)

In [13]:
pred_train, pred_test = makePred_eq_priors_est_var(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,est_var_eq_prior)
result2 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train,test_data_eq_prior,test_count_eq_prior,pred_test)

In [14]:
pred_train, pred_test = makePred_est_priors_est_var(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,est_var_eq_prior,est_prior_eq_prior)
result3 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train,test_data_eq_prior,test_count_eq_prior,pred_test)

In [15]:
pred_train, pred_test = makePred_est_priors_eq_var(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,est_var_eq_prior)
result4 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train,test_data_eq_prior,test_count_eq_prior,pred_test)

In [16]:
table = pd.DataFrame({"eq_pr_eq_var":result1[0],
                      "eq_pr_est_var":result2[0],
                      "est_pr_est_var":result3[0],
                      "est_pr_eq_var":result4[0]})
print("train data with equal priors")
table

train data with equal priors


,eq_pr_eq_var,eq_pr_est_var,est_pr_eq_var,est_pr_est_var
0,0.993913,0.787867,0.987029,0.0
1,0.781047,0.505412,0.742472,1.0
2,0.465121,0.685593,0.494451,0.0
3,0.982224,0.546308,0.968185,0.0
4,0.275883,0.627507,0.308154,0.0


In [17]:
table = pd.DataFrame({"eq_pr_eq_var":result1[1],
                      "eq_pr_est_var":result2[1],
                      "est_pr_est_var":result3[1],
                      "est_pr_eq_var":result4[1]})
print("test data with equal priors")
table

test data with equal priors


,eq_pr_eq_var,eq_pr_est_var,est_pr_eq_var,est_pr_est_var
0,0.992890,0.800406,0.988827,0.0
1,0.771127,0.498994,0.721831,1.0
2,0.468975,0.684945,0.498316,0.0
3,0.987406,0.545592,0.970277,0.0
4,0.273370,0.617484,0.305205,0.0


In [18]:
pred_train, pred_test = makePred_eq_priors_eq_var(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior)
result1 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train,test_data_noneq_prior,test_count_noneq_prior,pred_test)

In [19]:
pred_train, pred_test = makePred_eq_priors_est_var(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,est_var_noneq_prior)
result2 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train,test_data_noneq_prior,test_count_noneq_prior,pred_test)

In [20]:
pred_train, pred_test = makePred_est_priors_est_var(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,est_var_noneq_prior,est_prior_noneq_prior)
result3 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train,test_data_noneq_prior,test_count_noneq_prior,pred_test)

In [21]:
pred_train, pred_test = makePred_est_priors_eq_var(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,est_var_noneq_prior)
result4 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train,test_data_noneq_prior,test_count_noneq_prior,pred_test)

In [22]:
table = pd.DataFrame({"eq_pr_eq_var":result1[0],
                      "eq_pr_est_var":result2[0],
                      "est_pr_est_var":result3[0],
                      "est_pr_eq_var":result4[0]})
print("train data with unequal priors")
table

train data with unequal priors


,eq_pr_eq_var,eq_pr_est_var,est_pr_eq_var,est_pr_est_var
0,0.993188,0.540773,0.981767,0.0
1,0.792058,0.562035,0.759220,0.0
2,0.470279,0.740637,0.514128,1.0
3,0.979317,0.760678,0.966875,0.0
4,0.293625,0.571116,0.315339,0.0


In [23]:
table = pd.DataFrame({"eq_pr_eq_var":result1[1],
                      "eq_pr_est_var":result2[1],
                      "est_pr_est_var":result3[1],
                      "est_pr_eq_var":result4[1]})
print("test data with unequal priors")
table

test data with unequal priors


,eq_pr_eq_var,eq_pr_est_var,est_pr_eq_var,est_pr_est_var
0,0.994186,0.536822,0.983527,0.0
1,0.786640,0.570040,0.759919,0.0
2,0.473046,0.736523,0.515499,1.0
3,0.987595,0.765106,0.971589,0.0
4,0.280310,0.533463,0.303589,0.0
